In [1]:
import pandas as pd
import numpy as np
import tabula
import re
pd.set_option('display.max_rows', 500)

In [2]:
dft_2021 = tabula.read_pdf('2021 Blog.pdf', pages='all', stream=True, pandas_options={'header': None})
dft_2022 = tabula.read_pdf('2022 12 Patch Day Blog V9.0.pdf', pages='all', stream=True, pandas_options={'header': None})
dft_2023 = tabula.read_pdf('2023 12 Patch Day Blog V2.0 (1).pdf', pages='all', stream=True, pandas_options={'header': None})

Dec 12, 2024 7:20:19 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Dec 12, 2024 7:20:20 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Dec 12, 2024 7:20:20 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Dec 12, 2024 7:20:20 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Dec 12, 2024 7:20:21 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Dec 12, 2024 7:20:21 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>


In [3]:
def newHeader(xdf):
    data_list = []
    header = xdf[0].iloc[0]
    xdf[0].columns = header
    xdf[0] = xdf[0].drop([0])
    xdf[0]["cve_id"] = xdf[0]["Title"].str.extract(r'(CVE-....-\d+)')
    for i in range(0, len(xdf)):
        if i == 0:
            data = xdf[i].iloc[1:]
        else:
            data = xdf[i]
        if data.shape[1] == header.shape[0]:
            data.columns = header
            xdf[i]["cve_id"] = xdf[i]["Title"].str.extract(r'(CVE-....-\d+)')
            data_list.append(data)
    return data_list

In [4]:
dftt_2021 = newHeader(dft_2021)
dftt_2022 = newHeader(dft_2022)
dftt_2023 = newHeader(dft_2023)

In [5]:
sap_2021 = pd.concat(dftt_2021)
sap_2022 = pd.concat(dftt_2022)
sap_2023 = pd.concat(dftt_2023)

In [6]:
# Pandas merge rows in a column 'Title'
# https://stackoverflow.com/questions/64141451/pandas-merge-rows-in-a-column
def mergeTitle(ydf):
    ydf['Title'] = ydf['Title'].astype(str)
    blocks = ydf['CVSS'].notna().cumsum()
    agg_dict = {col:' '.join if col=='Title' else 'first' for col in ydf}
    df_t = ydf.groupby(blocks).agg(agg_dict).reset_index(drop=True)
    df_t.dropna(inplace=True)
    df_t = df_t[(df_t['Note#'] != 'Note#')]
    df_t['Note#'] = df_t['Note#'].astype(int)
    df_t.reset_index(drop=True, inplace=True)
    return df_t

In [7]:
sap_2021_notes = mergeTitle(sap_2021)
sap_2022_notes = mergeTitle(sap_2022)
sap_2023_notes = mergeTitle(sap_2023)

In [8]:
#sap_2023_notes

In [9]:
# SAP Notes 2024
#sap_2024_notes = pd.read_csv('sap_notes_2024.csv')

In [10]:
sap_202401 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/january-2024.html',
                 flavor='html5lib')
sap_202402 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/february-2024.html',
                 flavor='html5lib')
sap_202403 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/march-2024.html',
                 flavor='html5lib')
sap_202404 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/april-2024.html',
                 flavor='html5lib')
sap_202405 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/may-2024.html',
                 flavor='html5lib')
sap_202406 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/june-2024.html',
                 flavor='html5lib')
sap_202407 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/july-2024.html',
                 flavor='html5lib')
sap_202408 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/august-2024.html',
                 flavor='html5lib')
sap_202409 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/september-2024.html',
                 flavor='html5lib')
sap_202410 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/october-2024.html',
                         flavor='html5lib')
sap_202411 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/november-2024.html',
                         flavor='html5lib')
sap_202412 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/december-2024.html',
                         flavor='html5lib')

In [11]:
sap_202411

[     Note#                                              Title Priority  CVSS
 0  3520281  [CVE-2024-47590] Cross-Site Scripting (XSS) vu...     High   8.8
 1  3483344  Update to Security Note released on July 2024 ...     High   7.7
 2  3335394  [CVE-2024-42372] Missing Authorization check i...   Medium   6.5
 3  3509619  [CVE-2024-47595] Local Privilege Escalation in...   Medium   6.3
 4  3393899  [CVE-2024-47592] Information Disclosure Vulner...   Medium   5.3
 5  3504390  [CVE-2024-47586] NULL Pointer Dereference vuln...   Medium   5.3
 6  3522953  [CVE-2024-47588] Information Disclosure vulner...   Medium   4.7
 7  3508947  [CVE-2024-47593] Information Disclosure Vulner...   Medium   4.3
 8  3498470  [CVE-2024-47587] Missing authorization check i...      Low   3.5
 9  3392049  Update to Security Note released on May 2024 P...      Low   3.5]

In [12]:
def etData1(x):
    _df = pd.DataFrame(x[0], columns=['Note#', 'Title', 'Severity', 'CVSS'])
    _df.rename(columns={'Severity': 'Priority'}, inplace=True)
    _df["cve_id"] = _df["Title"].str.extract(r'(CVE-....-\d+)')
    return _df

def etData2(x):
    _df = pd.DataFrame(x[0], columns=['Note#', 'Title', 'Priority', 'CVSS'])
    _df["cve_id"] = _df["Title"].str.extract(r'(CVE-....-\d+)')
    return _df

In [13]:
sap_202401_df = etData1(sap_202401)
sap_202402_df = etData1(sap_202402)
sap_202403_df = etData1(sap_202403)
sap_202404_df = etData1(sap_202404)
sap_202405_df = etData1(sap_202405)
sap_202406_df = etData1(sap_202406)
sap_202407_df = etData2(sap_202407)
sap_202408_df = etData2(sap_202408)
sap_202409_df = etData2(sap_202409)
sap_202410_df = etData2(sap_202410)
sap_202411_df = etData2(sap_202411)
sap_202412_df = etData2(sap_202412)

In [14]:
sap_2024_ls = [sap_202401_df,sap_202402_df,sap_202403_df,sap_202404_df,sap_202405_df,sap_202406_df,sap_202407_df,sap_202408_df,sap_202409_df,sap_202410_df,sap_202411_df,sap_202412_df]
sap_2024_df = pd.concat(sap_2024_ls)

In [15]:
sap_2024_notes = sap_2024_df.dropna()
#sap_2024_notes = pd.read_csv('sap_notes_2024.csv')

In [33]:
# start extract cve_ids string for terminal commands
# Mejorar y crear un file para flag del commando.
l_sap_cve = sap_2021_notes.cve_id.to_list()

In [34]:
clean_sap_cve = [x for x in l_sap_cve if str(x) != 'nan']

In [35]:
#cleanedList

In [36]:
string_list = [str(element) for element in clean_sap_cve]
delimiter = ","
result_string_cve = delimiter.join(string_list)

In [37]:
result_string_cve

'CVE-2021-21446,CVE-2020-6307,CVE-2020-6224,CVE-2021-21445,CVE-2021-21447,CVE-2020-6256,CVE-2020-26816,CVE-2021-21448,CVE-2021-21469,CVE-2021-21449,CVE-2021-21467,CVE-2021-21470,CVE-2021-21477,CVE-2021-21465,CVE-2020-26832,CVE-2021-21446,CVE-2021-21472,CVE-2019-0337,CVE-2021-21444,CVE-2021-21476,CVE-2021-21478,CVE-2019-0388,CVE-2021-21474,CVE-2021-21475,CVE-2020-6207,CVE-2021-21480,CVE-2021-21481,CVE-2021-21484,CVE-2021-21486,CVE-2021-21488,CVE-2021-21487,CVE-2020-6308,CVE-2021-21491,CVE-2021-27592,CVE-2021-27585,CVE-2020-6316,CVE-2021-27596,CVE-2021-27602,CVE-2021-21481,CVE-2021-21482,CVE-2021-21483,CVE-2020-26832,CVE-2021-27608,CVE-2021-21485,CVE-2021-27598,CVE-2021-27603,CVE-2021-27599,CVE-2021-27604,CVE-2021-27600,CVE-2021-27601,CVE-2021-21491,CVE-2021-27609,CVE-2021-21492,CVE-2021-27605,CVE-2021-27602,CVE-2021-21466,CVE-2021-27611,CVE-2021-27616,CVE-2021-27613,CVE-2021-27619,CVE-2021-27617,CVE-2021-21491,CVE-2021-27609,CVE-2021-27612,CVE-2021-27602,CVE-2021-27610,CVE-2021-27635,CV

In [38]:
# end extract cve_ids

In [39]:
### Crear DataFrame de output Sploitscan
def dataframeSplotscan(file_json):
    data = pd.DataFrame(columns = ['cve_id', 'dateUpdated', 'descriptions', 'product_l', 'epss_l', 'percentile', 'priority_l', 'cweId'])
    dict_list = []
    sap_sp = pd.read_json(f'{file_json}', typ='series')
    for i in sap_sp:
        if 'problemTypes' in i['CVE Data']['containers']['cna'].keys():            
            if 'cweId' in i['CVE Data']['containers']['cna']['problemTypes'][0]['descriptions'][0].keys():
                cweId = i['CVE Data']['containers']['cna']['problemTypes'][0]['descriptions'][0]['cweId']
            else:
                cweId = i['CVE Data']['containers']['cna']['problemTypes'][0]['descriptions'][0]['description']
        else:
            cweId = None
        if len(i['EPSS Data']['data']) == 1:
            epss_l = i['EPSS Data']['data'][0]['epss']
            percentile = i['EPSS Data']['data'][0]['percentile']
        else:
            epss_l = None
            percentile = None
        if 'datePublished' in i['CVE Data']['cveMetadata']:
            datePublished = i['CVE Data']['cveMetadata']['datePublished']
        else:
            datePublished = None
        if 'references' in i['CVE Data']['containers']['cna'].keys():            
            note_id = re.findall('[2,3]{1}[0-9]{6}', str(i['CVE Data']['containers']['cna']['references'][0]['url']))
        else:
            note_id = None
        row_list = {'cve_id': i['CVE Data']['cveMetadata']['cveId'],
                    #'datePublished': i['CVE Data']['cveMetadata']['datePublished'],
                    'datePublished': datePublished,
                    'dateUpdated': i['CVE Data']['cveMetadata']['dateUpdated'],
                    'descriptions': i['CVE Data']['containers']['cna']['descriptions'][0]['value'],
                    'product_l': i['CVE Data']['containers']['cna']['affected'][0]['product'],
                    #'epss_l': i['EPSS Data']['data'][0]['epss'],
                    #'benchmark': i['VulnCheck Data']['_benchmark'],
                    'epss_l': epss_l,
                    'percentile': percentile,
                    'priority_l': i['Priority']['Priority'],
                    'cweId': cweId,
                    'note_id': str(note_id)}
        dict_list.append(row_list)

    data = pd.DataFrame.from_dict(dict_list)
    return(data)

In [40]:
#especial 2021 sap notes
#sp_sap_cve_2021 = dataframeSplotscan('sap_history_data_curate/sp_2021_last.json')

In [41]:
sp_sap_cve_2021 = dataframeSplotscan('sap_history_data_curate/sp_2021_last.json')
sp_sap_cve_2021.drop_duplicates(subset=['cve_id'], inplace=True)
sp_sap_cve_2022 = dataframeSplotscan('sap_history_data_curate/sp_2022_last.json')
sp_sap_cve_2022.drop_duplicates(subset=['cve_id'], inplace=True)
sp_sap_cve_2023 = dataframeSplotscan('sap_history_data_curate/sp_2023_last.json')
sp_sap_cve_2023.drop_duplicates(subset=['cve_id'], inplace=True)
sp_sap_cve_2024 = dataframeSplotscan('sap_history_data_curate/sp_2024_last.json')
sp_sap_cve_2024.drop_duplicates(subset=['cve_id'], inplace=True)

In [42]:
cp_sap_cve_2021 = pd.read_csv('sap_history_data_curate/cvep_2021.csv')
cp_sap_cve_2022 = pd.read_csv('sap_history_data_curate/cvep_2022.csv')
cp_sap_cve_2023 = pd.read_csv('sap_history_data_curate/cvep_2023.csv')
cp_sap_cve_2024 = pd.read_csv('sap_history_data_curate/cvep_2024.csv')

In [43]:
cp_sap_cve_2021.drop_duplicates(subset=['cve_id'], inplace=True)
cp_sap_cve_2022.drop_duplicates(subset=['cve_id'], inplace=True)
cp_sap_cve_2023.drop_duplicates(subset=['cve_id'], inplace=True)
cp_sap_cve_2024.drop_duplicates(subset=['cve_id'], inplace=True)

In [44]:
sap_cve_2023 = sap_2023_notes.merge(sp_sap_cve_2023, on=['cve_id'])
sap_cve_2023 = sap_cve_2023.merge(cp_sap_cve_2023, on=['cve_id'])
sap_cve_2023['sap_note_year'] = '2023'

In [45]:
sap_cve_2022 = sp_sap_cve_2022.merge(sap_2022_notes, on=['cve_id'])
sap_cve_2022 = sap_cve_2022.merge(cp_sap_cve_2022, on=['cve_id'])
sap_cve_2022['sap_note_year'] = '2022'

In [46]:
sap_cve_2021 = sp_sap_cve_2021.merge(sap_2021_notes, on=['cve_id'])
sap_cve_2021 = sap_cve_2021.merge(cp_sap_cve_2021, on=['cve_id'])
sap_cve_2021['sap_note_year'] = '2021'

In [47]:
sap_cve_2024 = sp_sap_cve_2024.merge(sap_2024_notes, on=['cve_id'])
sap_cve_2024 = sap_cve_2024.merge(cp_sap_cve_2024, on=['cve_id'])
sap_cve_2024['sap_note_year'] = '2024'

In [48]:
sap_cve_df = pd.concat([sap_cve_2021,sap_cve_2022,sap_cve_2023,sap_cve_2024])

In [49]:
sap_cve_df.loc[(sap_cve_df['Priority'] == 'Hot') | (sap_cve_df['Priority'] == 'HotNews') | (sap_cve_df['Priority'] == 'Very High'), 'Priority'] = 'Hot News'
#sap_cve_df.loc[(sap_cve_df['Priority'] == 'Hot') | (sap_cve_df['Priority'] == 'HotNews'), 'Priority'] = 'Hot News'

sap_cve_df.loc[(sap_cve_df['cweId'] == 'Cross-Site Scripting') | (sap_cve_df['cweId'] == 'Cross Site Scripting') | (sap_cve_df['cweId'] == "Cross-Site Scripting (XSS)"), 'cweId'] = "CWE-79"

sap_cve_df.loc[(sap_cve_df['cweId'] == 'Missing Authorization check') | (sap_cve_df['cweId'] == 'Missing Authorization') | (sap_cve_df['cweId'] == 'Missing Authorization Check'), 'cweId'] = "CWE-862"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21484'),'cweId'] = "CWE-863"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2023-30533'),'cweId'] = "CWE-1321"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2022-35737'),'cweId'] = "CWE-129"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2023-44487'),'cweId'] = "CWE-400"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-6308'),'cweId'] = "CWE-918"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-6207'),'cweId'] = "CWE-306"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33690'),'cweId'] = "CWE-918"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-38163'),'cweId'] = "CWE-78"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-44235'),'cweId'] = "CWE-78"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-37531'),'cweId'] = "CWE-78"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33663'),'cweId'] = "CWE-74"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2024-33007'),'cweId'] = "CWE-79"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27608'),'cweId'] = "CWE-428"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27635'),'cweId'] = "CWE-112"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27635'),'cweId'] = "CWE-112"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27617'),'cweId'] = "CWE-112"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-40499'),'cweId'] = "CWE-94"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVEp-2021-27611'),'cweId'] = "CWE-94"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21466'),'cweId'] = "CWE-94"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27602'),'cweId'] = "CWE-94"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-44231'),'cweId'] = "CWE-94"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21480'),'cweId'] = "CWE-94"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-10683'),'cweId'] = "CWE-611"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21444'),'cweId'] = "CWE-1021"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2019-17495'),'cweId'] = "CWE-352"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-44151'),'cweId'] = "CWE-330"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2013-3587'),'cweId'] = 'CWE-200'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2019-0388'),'cweId'] = 'CWE-290'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-26816'),'cweId'] = 'CWE-312'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-6215'),'cweId'] = 'CWE-601'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-6224'),'cweId'] = 'CWE-532'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21445'),'cweId'] = 'CWE-444'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21449'),'cweId'] = 'CWE-119'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21465'),'cweId'] = 'CWE-89'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21469'),'cweId'] = 'CWE-200'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21470'),'cweId'] = 'CWE-611'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21472'),'cweId'] = 'CWE-306'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21474'),'cweId'] = 'CWE-326'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21475'),'cweId'] = 'CWE-22'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21476'),'cweId'] = 'CWE-601'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21477'),'cweId'] = 'CWE-94'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21478'),'cweId'] = 'CWE-601'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21488'),'cweId'] = 'CWE-502'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21491'),'cweId'] = 'CWE-601'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27610'),'cweId'] = 'CWE-287'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27612'),'cweId'] = 'CWE-601'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27638'),'cweId'] = 'CWE-20'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33672'),'cweId'] = 'CWE-116'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33676'),'cweId'] = 'CWE-862'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33685'),'cweId'] = 'CWE-22'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33687'),'cweId'] = 'CWE-200'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33688'),'cweId'] = 'CWE-89'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-38150'),'cweId'] = 'CWE-312'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-38176'),'cweId'] = 'CWE-89'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-38177'),'cweId'] = 'CWE-476'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-40497'),'cweId'] = 'CWE-668'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-42064'),'cweId'] = 'CWE-89'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-42068'),'cweId'] = 'CWE-20'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-44232'),'cweId'] = 'CWE-22'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2023-0215'),'cweId'] = 'CWE-416'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-6369'),'cweId'] = 'CWE-798'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-13936'),'cweId'] = 'CWE-94'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21446'),'cweId'] = 'CWE-400'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21482'),'cweId'] = 'CWE-200'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21483'),'cweId'] = 'CWE-200'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21485'),'cweId'] = 'CWE-200'

In [50]:
sap_cve_df.to_csv('sap_cve_last_202412_all.csv', index=False)

In [53]:
sap_cve_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 677 entries, 0 to 159
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cve_id         677 non-null    object 
 1   datePublished  677 non-null    object 
 2   dateUpdated    677 non-null    object 
 3   descriptions   677 non-null    object 
 4   product_l      677 non-null    object 
 5   epss_l         677 non-null    object 
 6   percentile     677 non-null    object 
 7   priority_l     677 non-null    object 
 8   cweId          677 non-null    object 
 9   note_id        677 non-null    object 
 10  Note#          677 non-null    int64  
 11  Title          677 non-null    object 
 12  Priority       677 non-null    object 
 13  CVSS           677 non-null    object 
 14  priority       677 non-null    object 
 15  epss           677 non-null    float64
 16  cvss           677 non-null    float64
 17  cvss_version   677 non-null    object 
 18  cvss_severity  

In [51]:
sap_cve_df.loc[(sap_cve_df['Priority'] == 'Medium')]['Priority'].count()

407